In [ ]:
!pip install selectolax 

# 1. Parsing HTML (allow 8 mins)
Task You will scrape and process simple html page located here candidateEvalData/webpage.html

Output A dataframe of 1 row and 7 columns where the columns are:

1. The name of the artist (Peter Doig)
2. The name of the painting (The Architect's Home in the Ravine)
3. Price realised in GBP (11 282 500)
4. Price realised in USD (6 370 908)
5. Estimates in GBP (10 000 000 , 15 000 000)
6. Estimate in USD (14 509 999 , 21 764999)
7. The url of the image of the painting
8. Saledate of the painting (2016-02-11)

In [1]:
#import modules
from lxml import html
import re
from selectolax.parser import HTMLParser
import requests
import pandas as pd
from datetime import datetime

#get html and tree
html_page_link = 'candidateEvalData/webpage.html'

response = open(html_page_link, 'r').read()

content = HTMLParser(response)
section = content.css_first('div#MainContentDetails')

# parse artist name
artist_name = section.css_first('div#MainContent h1#main_center_0_lblLotPrimaryTitle').text().strip()

pattern= re.search(r"\(.*\)", artist_name)
artist_name_fin = re.sub(pattern, '', artist_name)


#parse painting name
name = section.css_first('h2#main_center_0_lblLotSecondaryTitle').text().strip()

#parse price GBP
gpb_price = section.css_first('div.price span#main_center_0_lblPriceRealizedPrimary').text().strip()

#parse price US
usd_price = section.css_first('div.column div#main_center_0_lblPriceRealizedSecondary').text().strip()


#parse price GBP est
estimated_gpd_price = section.css_first('div.price span#main_center_0_lblPriceEstimatedPrimary').text().strip()

#parse price US est
price = section.css_first('div.price span#main_center_0_lblPriceEstimatedSecondary').text().strip()
us_price_price= price.replace('(', '').replace(')', '')

#image link

image_link = content.css_first('li.box-link img#imgLotImage').attrs['src']


# 2. Regex
A common regex was found all rows in the dataframe of the expressions
- These search result was extracted using the extract_dimensions function
- The results were parsed into their correct format using the change_fraction,
  inch_cm, and format_results functions
- These function is put into one function  get_dimensions and can be applied using 
the __.apply__ method on dataframes 
 

In [ ]:
import pandas as pd
from fractions import Fraction


dim_df = pd.read_csv("candidateEvalData/dim_df_correct.csv")

def extract_dimensions(text: str)-> list:
    """Extracts the dimensions of a text.

    Args: Text containing dimensions to be extracted
    Returns: List of dimensions
    """
    #Subs dimensions that are within parenthesis for empty strings
    sub_com =re.compile(r"\(.*\)")
    tex_sub = re.sub(sub_com, '' , text)

    #Return matches for the regex expression
    pattern = re.compile(r"(\d+\s*\d[.,/]?\d*)\s*(cm|inch|in)?")
    com_list = pattern.findall(tex_sub)

    return com_list


def change_fractions(match_list:list)-> list:
    """Changes the fractions in the text to decimals.

    Args: List of matches

    """
    for i in range(len(match_list)):
        match_list[i] = list(match_list[i]) #tuples are immutable
        match_list[i][0] = round(float(sum(Fraction(s) for s in match_list[i][0].split())), 1)

    return match_list


def inch_to_cm(results:list)-> list:
    """Converts inches to centimeters.

    Args: List of matches
    """
    dimension_list = [x[1] for x in results]

    if "in" in dimension_list:
        results_cm = [round(x[0]*2.54, 1) for x in results]
        return results_cm

    else:
        results_cm =  [x[0] for x in results]

        return results_cm
    

def format_results(results_cm:list)-> list:
    """Makes sure a list of three elements are parsed.

    Args: Results in cm
    """

    if len(results_cm) == 3:
        return results_cm

    elif len(results_cm) == 2:
        results_cm.append(None)
        return results_cm


def get_dimensions(text: str)-> list:
    """Final function that incorporates the others
    """

    list1 =extract_dimensions(text)
    list_ini = change_fractions(list1)
    list_inte = inch_to_cm(list_ini)
    list_formated = format_results(list_inte)

    return list_formated



## Inner Join
This is a kind of join that selects records that have matching values in both tables.
![Inner Join](data/inner.PNG)

## Right Join
This join returns all the rows of the table on the right side of the join and matching rows for the table on the left side of the join.

![Right Join](data/right.PNG)

## Left Join
This join returns all the rows of the table on the left side of the join and matching rows for the table on the right side of the join.

![Left Join](data/left.PNG)

## Full Join
This join returns all records when there is a match in left or right table records.

![Full Join](data/full.PNG)



```SQL
SELECT origin, count(origin) as numFlight

FROM
(SELECT f.arr_time, f.origin, f.dest, a.name
FROM flight f
JOIN airline a
ON f.carrier=a.carrier
WHERE name like '%JetBlue%'
ORDER by 2 asc) a

GROUP by origin
HAVING numFlight>10000
ORDER by origin ASC
```